# Конвертация в формат Yolo

In [1]:
! pip install pylabel

  Obtaining dependency information for pylabel from https://files.pythonhosted.org/packages/54/5a/30e4cc2d2df68d2e2375863afe4a9de54c90042fb29378f096d1d8095396/pylabel-0.1.55-py3-none-any.whl.metadata
  Obtaining dependency information for bbox-visualizer from https://files.pythonhosted.org/packages/f8/37/55a8724dec292a05300e7d2038ca9f6f478708e64bf42a01a21a51622ffd/bbox_visualizer-0.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/a1/a6/c5b78606743a1f28eae8f11973de6613a5ee87366796583fb74c67d54939/scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for jupyter-bbox-widget from https://files.pythonhosted.org/packages/b4/e4/bfab95d47c97ec1316ca78f11ae02b4bf9d877b16014f4151d6467fbc448/jupyter_bbox_widget-0.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for anywidget>=0.9.0 from https://files.pythonhosted.org/packages/2b/f0/09a30ca0551af20c7cefa7464b7ccb6f5407a


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from pylabel import importer

PATH_TO_ANNOTATIONS = 'F:/GitHub/datasets/TableBlank/TableBank/Detection/annotations'

dataset = importer.ImportVOC(path=PATH_TO_ANNOTATIONS)
dataset.export.ExportToYoloV5()

Importing VOC files...: 100%|██████████| 6/6 [00:00<00:00, 2998.43it/s]
Exporting YOLO files...: 0it [00:00, ?it/s]


['training\\dataset.yaml']

In [8]:
import json
import os
import shutil
from tqdm import tqdm

ANNOTATIONS_DIR = "F:/GitHub/datasets/TableBlank/TableBank/Detection/annotations"
IMAGES_DIR = "F:/GitHub/datasets/TableBlank/TableBank/Detection//images"
OUTPUT_DIR = "dataset_yolo"
SPLITS = ["train", "val", "test"]
SOURCE = "latex"  # можно поменять на 'word' или объединить оба

for split in SPLITS:
    os.makedirs(os.path.join(OUTPUT_DIR, "images", split), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "labels", split), exist_ok=True)

for split in SPLITS:
    print(f"Обрабатывается: {split}")
    
    json_path = os.path.join(ANNOTATIONS_DIR, f"tablebank_{SOURCE}_{split}.json")
    with open(json_path, "r") as f:
        data = json.load(f)

    # Построим словарь image_id -> info
    image_dict = {img["id"]: img for img in data["images"]}

    # Разделим аннотации по изображениям
    grouped_annotations = {}
    for ann in data["annotations"]:
        img_id = ann["image_id"]
        grouped_annotations.setdefault(img_id, []).append(ann)

    # Обрабатываем каждое изображение
    for img_id, image_info in tqdm(image_dict.items()):
        file_name = image_info["file_name"]
        img_w, img_h = image_info["width"], image_info["height"]

        # Копируем изображение
        src_img_path = os.path.join(IMAGES_DIR, file_name)
        dst_img_path = os.path.join(OUTPUT_DIR, "images", split, file_name)
        if os.path.exists(src_img_path):
            shutil.copyfile(src_img_path, dst_img_path)
        else:
            print(f"Пропущено (нет изображения): {src_img_path}")
            continue

        # Записываем аннотации в YOLO-формате
        label_path = os.path.join(
            OUTPUT_DIR, "labels", split, os.path.splitext(file_name)[0] + ".txt"
        )
        with open(label_path, "w") as f:
            for ann in grouped_annotations.get(img_id, []):
                x, y, w, h = ann["bbox"]
                x_center = (x + w / 2) / img_w
                y_center = (y + h / 2) / img_h
                w_norm = w / img_w
                h_norm = h / img_h
                f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

print("Конвертация завершена!")

# ==== Создание data.yaml ====
yaml_path = os.path.join(OUTPUT_DIR, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(f"""\
train: {os.path.abspath(os.path.join(OUTPUT_DIR, 'images/train'))}
val: {os.path.abspath(os.path.join(OUTPUT_DIR, 'images/val'))}

nc: 1
names: ['table']
""")

print(f"data.yaml создан по пути: {yaml_path}")


Обрабатывается: train


100%|██████████| 187199/187199 [16:38<00:00, 187.42it/s]


Обрабатывается: val


100%|██████████| 7265/7265 [00:38<00:00, 189.47it/s]


Обрабатывается: test


100%|██████████| 5719/5719 [00:30<00:00, 187.16it/s]

Конвертация завершена!
data.yaml создан по пути: dataset_yolo\data.yaml


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

model.train(
    data='table_data.yaml',  # data.yaml
    epochs=50,               # кол-во эпох 
    imgsz=640,               # размер входного изображения 
    batch=16,                # размер batch
    name='table_detection',  # подпапка для логов (runs/detect/table_detection/)
    device=0
)